# Import Libraries

In [7]:
import kagglehub
import pandas as pd
import numpy as np
import os
import zipfile
import matplotlib.pyplot as plt
from scipy import stats
from kagglehub import KaggleDatasetAdapter


# Import the Dataset

Dataset Info

URL: https://www.kaggle.com/datasets/katerynameleshenko/ai-index/data

Indicator Documentation: https://www.tortoisemedia.com/data/global-ai#rankings

Tesis Title: Development of a Model to Estimate the Probability of AI Job Substitution in Zapopan

In [8]:
# --- Descargar dataset ---
path = kagglehub.dataset_download("katerynameleshenko/ai-index")
print("Dataset descargado en:", path)

# --- Listar archivos dentro de la carpeta descargada ---
print("Archivos disponibles:", os.listdir(path))

# --- Si viene en ZIP, lo descomprimimos ---
zip_files = [f for f in os.listdir(path) if f.endswith(".zip")]
if zip_files:
    zip_path = os.path.join(path, zip_files[0])
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path)
    print("Archivos después de extraer:", os.listdir(path))

# --- Buscar CSVs extraídos ---
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta del dataset.")

# --- Tomamos el primer CSV ---
file_path = os.path.join(path, csv_files[0])
print("Usando archivo:", file_path)

# --- Cargar con pandas ---
df = pd.read_csv(file_path)

# --- Mostrar primeras filas ---
print("Primeros 5 registros:")
print(df.head())

100%|██████████| 2.38k/2.38k [00:00<00:00, 1.45MB/s]

Extracting files...
Dataset descargado en: C:\Users\carlo\.cache\kagglehub\datasets\katerynameleshenko\ai-index\versions\1
Archivos disponibles: ['AI_index_db.csv']
Usando archivo: C:\Users\carlo\.cache\kagglehub\datasets\katerynameleshenko\ai-index\versions\1\AI_index_db.csv
Primeros 5 registros:
                    Country  Talent  Infrastructure  Operating Environment  \
0  United States of America  100.00           94.02                  64.56   
1                     China   16.51          100.00                  91.57   
2            United Kingdom   39.65           71.43                  74.65   
3                    Canada   31.28           77.05                  93.94   
4                    Israel   35.76           67.58                  82.44   

   Research  Development  Government Strategy  Commercial  Total score  \
0    100.00       100.00                77.39      100.00       100.00   
1     71.42        79.97                94.87       44.02        62.92   
2     36.5